In [140]:
import pandas as pd
import numpy as np


In [159]:
import mysql.connector

def load_stockdata(symbol):
    serverName = "localhost"
    cnx = mysql.connector.connect(user='michael', password='Password2017',
                                  host=serverName, database='stock_market')
    cursor = cnx.cursor()
    
    query = \
    "SELECT stock.*, bb.`sma20` AS bb_middle, bb.`band_upper` AS bb_upper, bb.`band_lower` AS bb_lower, \
    kc.`ema20` AS kc_middle, kc.`atr2_upper` AS kc_2_upper, kc.`atr2_lower` AS kc_2_lower, \
    kc.`atr3_upper` AS kc_3_upper, kc.`atr3_lower` AS kc_3_lower, \
    kc.`atr1_upper` AS kc_1_upper, kc.`atr1_lower` AS kc_1_lower \
    FROM \
    ( SELECT * FROM (SELECT symbol, effective_date, `adj_open`, `adj_high`, `adj_low`, `adj_close`, `adj_volume` FROM `market.stock_price` \
      WHERE symbol = '{0}' ORDER BY effective_date DESC LIMIT 300) sub ORDER BY `effective_date` ASC ) stock \
    LEFT JOIN \
    `idc.bollinger_bands` AS bb ON bb.symbol = stock.symbol AND bb.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.keltner_channels` AS kc ON kc.symbol = stock.symbol AND kc.effective_date = stock.effective_date".format(symbol)

    cursor.execute(query)
    dfsql = pd.DataFrame(cursor.fetchall(), dtype=np.float)
    dfsql.columns = cursor.column_names
    dfsql.set_index('effective_date', inplace=True)
    cursor.close()
    return dfsql


In [148]:
def drawChart(df):
    ax = df[["close"]].plot.line(grid='true', figsize=(18,10), title='Close chart', 
                                          marker=".", markersize=10, fontsize=18)
    ax.set_xlabel('tenor', fontsize=18)

In [149]:
import plotly.plotly as py
import plotly.graph_objs as go
from datetime import datetime

import plotly
plotly.offline.init_notebook_mode(connected=True) 

In [150]:
def drawInterChart(df):
    trace = go.Ohlc(x=df.date, open=df.open, high=df.high, low=df.low, close=df.close)
    data = [trace]
    layout = {
        'title': 'Stock Chart',
        'yaxis': {'title': 'Price'},
        'shapes': [{
            'x0': '2016-12-09', 'x1': '2016-12-09',
            'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
            'line': {'color': 'rgb(30,30,30)', 'width': 1}
        }],
        'annotations': [{
            'x': '2016-12-09', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
            'showarrow': False, 'xanchor': 'left',
            'text': 'Increase Period Begins'
        }]
    }
    fig = dict(data=data, layout=layout)
    
    # 
    # embbeded chart
    #plotly.offline.iplot(fig)
    
    # standalone chart
    plotly.offline.plot(fig)
    
    # online chart 
    # py.iplot(fig, filename='appl-recession-candlestick')

In [164]:
df = load_stockdata('MSFT')
df.head()


,symbol,adj_open,adj_high,adj_low,adj_close,adj_volume,bb_middle,bb_upper,bb_lower,kc_middle,kc_2_upper,kc_2_lower,kc_3_upper,kc_3_lower,kc_1_upper,kc_1_lower
effective_date,,,,,,,,,,,,,,,,
2018-02-15,MSFT,91.210,92.72,90.6200,92.66,27407914.0,91.2420,95.952179,86.531821,90.287573,95.298923,85.276223,97.804598,82.770548,92.793248,87.781898
2018-02-14,MSFT,88.510,90.99,88.4100,90.81,34314101.0,91.1140,95.802170,86.425830,90.037844,95.111606,84.964082,97.648487,82.427201,92.574725,87.500963
2018-02-13,MSFT,88.930,90.00,87.8000,89.83,26200053.0,91.0805,95.786422,86.374578,89.956565,95.023693,84.889437,97.557257,82.355873,92.490129,87.423001
2018-02-12,MSFT,88.735,89.78,87.9295,89.13,35501368.0,91.0065,95.833732,86.179268,89.969887,95.088333,84.851441,97.647556,82.292218,92.529110,87.410664
2018-02-09,MSFT,86.300,88.93,83.8300,88.10,57408480.0,91.0290,95.825131,86.232869,90.058296,95.285776,84.830816,97.899516,82.217076,92.672036,87.444556


In [160]:
def multi_chart(df):
    
    style_bollingerbands_middle = dict( color = ('rgb(22, 96, 167)'), width = 1, dash = 'dot')
    style_bollingerbands_upper = dict( color = ('rgb(22, 96, 167)'), width = 1, dash = 'line')
    style_bollingerbands_lower = dict( color = ('rgb(91, 154, 255)'), width = 1, dash = 'line')

    style_kc_middle = dict( color = ('rgb(247, 94, 39)'), width = 1, dash = 'dot')
    style_kc2_upper = dict( color = ('rgb(247, 94, 39)'), width = 1, dash = 'line')
    style_kc2_lower = dict( color = ('rgb(244, 118, 73)'), width = 1, dash = 'line')

    style_kc3_upper = dict( color = ('rgb(46, 155, 53)'), width = 1, dash = 'line')
    style_kc3_lower = dict( color = ('rgb(68, 206, 77)'), width = 1, dash = 'line')

    style_kc1_upper = dict( color = ('rgb(244, 78, 66)'), width = 1, dash = 'line')
    style_kc1_lower = dict( color = ('rgb(198, 70, 61)'), width = 1, dash = 'line')

    trace_candle = go.Candlestick(x=df.index, open=df.adj_open, high=df.adj_high, low=df.adj_low, close=df.adj_close, 
                         showlegend=False, name='close price')

    trace_ohlc = go.Ohlc(x=df.index, open=df.adj_open, high=df.adj_high, low=df.adj_low, close=df.adj_close, 
                         showlegend=False, name='close price',
                         increasing=dict(line=dict(color= '#b1e2b6')),
                         decreasing=dict(line=dict(color= '#a89a95'))
                        )
    
    trace_bb_middle = go.Scatter(x=df.index, y=df.bb_middle, name='BB Middle', legendgroup='Bollinger Bands',
                                 showlegend=False, opacity=0.5,
                                 line = style_bollingerbands_middle
                                )
    trace_bb_upper = go.Scatter(x=df.index, y=df.bb_upper, name='BB Upper', legendgroup='Bollinger Bands',
                                line = style_bollingerbands_upper
                               )
    trace_bb_lower = go.Scatter(x=df.index, y=df.bb_lower, name='BB Lower', legendgroup='Bollinger Bands',
                                line = style_bollingerbands_lower
                               )
    
    trace_kc_middle = go.Scatter(x=df.index, y=df.kc_middle, name='KC Middle', legendgroup='Keltner Channels 2ATR',
                                 showlegend=False, opacity=0.5,
                                 line = style_kc_middle
                                )
    trace_kc_2atr_upper = go.Scatter(x=df.index, y=df.kc_2_upper, name='KC 2ATR Upper', legendgroup='Keltner Channels 2ATR',
                                     line = style_kc2_upper
                                    )
    trace_kc_2atr_lower = go.Scatter(x=df.index, y=df.kc_2_lower, name='KC 2ATR Lower', legendgroup='Keltner Channels 2ATR',
                                     line = style_kc2_lower
                                    )

    trace_kc_3atr_upper = go.Scatter(x=df.index, y=df.kc_3_upper, name='KC 3ATR Upper', legendgroup='Keltner Channels 3ATR',
                                     line = style_kc3_upper
                                    )
    trace_kc_3atr_lower = go.Scatter(x=df.index, y=df.kc_3_lower, name='KC 3ATR Lower', legendgroup='Keltner Channels 3ATR',
                                     line = style_kc3_lower
                                    )
    trace_kc_1atr_upper = go.Scatter(x=df.index, y=df.kc_1_upper, name='KC 1ATR Upper', legendgroup='Keltner Channels 1ATR',
                                     line = style_kc1_upper
                                    )
    trace_kc_1atr_lower = go.Scatter(x=df.index, y=df.kc_1_lower, name='KC 1ATR Lower', legendgroup='Keltner Channels 1ATR',
                                     line = style_kc1_lower
                                    )

    # create volume bar chart on the bottom
    tarce_volume = go.Bar(x=df.index, y=df.adj_volume, name='volume', showlegend=False, marker=dict(color='rgb(87, 91, 130)'),
                          yaxis = 'y2'
                         )
    
    data = [trace_ohlc, trace_bb_middle, trace_bb_upper, trace_bb_lower, trace_kc_1atr_lower, trace_kc_1atr_upper,
            trace_kc_middle, trace_kc_2atr_upper, trace_kc_2atr_lower, trace_kc_3atr_upper, trace_kc_3atr_lower, tarce_volume]
    
    layout = go.Layout(
        title = 'John Carter Squeeze',
        xaxis = dict(rangeslider = dict(visible = False)),
        yaxis = dict(domain=[0.2,1]),
        yaxis2 = dict(domain=[0,0.2], visible=False),
        legend = dict(x=0.05, y=1.0)
    )

    fig = go.Figure(data=data, layout=layout)
    # standalone chart
    plotly.offline.plot(fig, "multi chart")


In [163]:
multi_chart(df)